# CD5 Results Cleanup and Analysis
This notebook cleans up the output of CD5 calculations and does some simple analysis.

##Log In

In [ ]:

!pip install dimcli -U --quiet

import dimcli
from dimcli.utils import *
import os,sys,time,json
from os.path import exists
import copy
import pandas as pd

print ("Logging in")
#Use dsl.ini file
dsl=dimcli.login()
dsl=dimcli.Dsl()


     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 240.6/240.6 kB 3.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.6/1.6 MB 11.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 51.1/51.1 kB 3.5 MB/s eta 0:00:00


Searching config file credentials for default 'live' instance..


Logging in
Dimcli - Dimensions API Client (v1.1)
Connected to: <https://nsf.dimensions.ai/api/dsl> - DSL v2.7
Method: dsl.ini file


##Input and Output Files

In [ ]:
original_grant_file="SSI2011.xlsx"
inputDataFile="out_all_SSI2011_final.xlsx"
outputDataFile="out_all_SSI2011_final_cleaned.xlsx"

##Remove Duplicates
Remove any duplicated entries.

In [ ]:
# Read files
grants=pd.read_excel(original_grant_file)
publications = pd.read_excel(inputDataFile,verbose=True)

#Drop duplicates and NaNs
pubs_cleaned=publications.drop_duplicates(subset='id')
pubs_cleaned=pubs_cleaned.dropna()

#Drop 0.0's
#See https://stackoverflow.com/questions/13851535/how-to-delete-rows-from-a-pandas-dataframe-based-on-a-conditional-expression
pubs_cleaned = pubs_cleaned.drop(pubs_cleaned[pubs_cleaned.CD5==0.0].index)

#Sort the file for later consistency.
pubs_cleaned=pubs_cleaned.sort_values('id',ascending=False)
pubs_cleaned=pubs_cleaned.reset_index(drop=True)
pubs_cleaned

##Add Missing Grant Information
Add the grant information for each publication

In [ ]:
# Define a function to get the grant supporting information.
def getGrantsForPublication(pub_id):
  data=dsl.query(f'''
    search publications where id in {json.dumps(pub_id)}
    return publications[id+supporting_grant_ids]
    limit 1000
  ''').as_dataframe()
  return data


In [ ]:
#We need to send a list of publications to DSL
pubs_cleaned_list=list(pubs_cleaned['id'])
pubs_and_grants=getGrantsForPublication(pubs_cleaned_list)

#Sort and reindex the output so that it is consistently ordered with pub_cleaned
pubs_and_grants=pubs_and_grants.sort_values('id',ascending=False)
pubs_and_grants=pubs_and_grants.reset_index(drop=True)
pubs_and_grants

In [ ]:
#Use the update method to update the supporting_grant_ids field
pubs_final=copy.deepcopy(pubs_cleaned)
pubs_final.update(pubs_and_grants)
pubs_final=pubs_final.reset_index(drop=True)
pubs_final
#pub.1011655136

,id,CD5,mCD5,supporting_grant_ids
0,pub.1141422850,-0.000395,-0.006318,"[grant.7914313, grant.7914317, grant.3131823]"
1,pub.1133062394,-0.000733,-0.007335,"[grant.6439018, grant.7735181, grant.3132605, ..."
2,pub.1130823891,-0.000549,-0.005493,"[grant.13218500, grant.7914317, grant.3131823]"
3,pub.1130663953,-0.002796,-0.027956,"[grant.3132612, grant.4603891, grant.3118244]"
4,pub.1127872465,-0.001856,-0.070518,"[grant.3132592, grant.3479738, grant.3004615, ..."
...,...,...,...,...
370,pub.1001807116,0.000451,0.048213,"[grant.3000154, grant.3004453]"
371,pub.1001307338,-0.000190,-0.002667,"[grant.3132582, grant.2519035, grant.2514627]"
372,pub.1000741367,-0.001706,-0.170577,"[grant.3120416, grant.3132582]"
373,pub.1000679613,-0.000197,-0.003352,"[grant.3132504, grant.3122151, grant.4057985]"


In [ ]:
def findDimensionGrantID(nsfGrantNumber):
  grant = dsl.query(f'''extract_grants(grant_number="{nsfGrantNumber}", funder_name="National Science Foundation")''',verbose=False).json
  grant_id = grant.get("grant_id")
  return grant_id

grantID_list=[]
for x in grants[0][:]:
  grantID=findDimensionGrantID(int(x))
  grantID_list.append(grantID)

grantID_list


In [ ]:
#df[df['A'].str.contains("hello")]
#df1[df1['col'].str.contains('foo', regex=False)]
testVal='grant.7914317'

# This gives the maximum value of mCD5 per grant. Some grants don't have any tested pubs
grant_count1=pd.DataFrame(columns=['GrantID','Maximum mCD5'])
for grantID in grantID_list:
  matchingPubs=pubs_final[pubs_final['supporting_grant_ids'].astype('string').str.contains(grantID)]
  if(len(matchingPubs['id'])>0):
    new_row={'GrantID':grantID,'Maximum mCD5':max(matchingPubs['mCD5'])}
    grant_count1.loc[len(grant_count1)]=new_row
    #print(grantID,max(matchingPubs['mCD5']))
grant_count1

,GrantID,Maximum mCD5
0,grant.3132622,-0.016101
1,grant.3125047,0.046815
2,grant.3132369,-0.069444
3,grant.3132582,-0.000291
4,grant.3129785,-0.010290
5,grant.3132612,-0.014208
6,grant.3132623,1.320717
7,grant.3132624,-0.021468
8,grant.3132419,-0.012906
9,grant.3132573,-0.117967


In [ ]:
# This counts the number of tested publications per grant.
grant_count2=pd.DataFrame(columns=['GrantID','Tested Publications'])
for grantID in grantID_list:
  matchingPubs=pubs_final[pubs_final['supporting_grant_ids'].astype('string').str.contains(grantID)]
  new_row={'GrantID':grantID,'Tested Publications':len(matchingPubs['id'])}
  #print(new_row)
  #df.loc[len(df)] = list_row
  grant_count2.loc[len(grant_count2)]=new_row
grant_count2

,GrantID,Tested Publications
0,grant.3132622,1
1,grant.3125047,9
2,grant.3130799,0
3,grant.3131143,0
4,grant.3132369,4
...,...,...
72,grant.3132592,7
73,grant.3001430,1
74,grant.3004453,38
75,grant.3132579,0


##Write the Cleaned File

In [ ]:
# Write the file
pubs_final.to_excel(outputDataFile,index=False)